In [17]:
import os
print("Current working directory:", os.getcwd())
print("Expected files in:", Path("./04a-risk_volume/norgate_continuous/parquet").resolve())

Current working directory: c:\TWS API\source\pythonclient\TradingIdeas\Futures
Expected files in: C:\TWS API\source\pythonclient\TradingIdeas\Futures\04a-risk_volume\norgate_continuous\parquet


In [18]:
from pathlib import Path

# Check if directory exists
dir_path = Path("./04a-risk_volume/norgate_continuous/parquet")
print(f"Directory exists: {dir_path.exists()}")

# List files
if dir_path.exists():
    files = list(dir_path.glob("*.parquet"))
    print(f"Number of files: {len(files)}")
    print(f"First 5 files: {[f.name for f in files[:5]]}")
    
    # Test loading one file
    if files:
        test_file = files[0]
        import pandas as pd
        df = pd.read_parquet(test_file)
        print(f"\nTest file: {test_file.name}")
        print(f"Has usd_risk_volume: {'usd_risk_volume' in df.columns}")
        print(f"Columns: {df.columns.tolist()}")

Directory exists: True
Number of files: 105
First 5 files: ['6A.parquet', '6B.parquet', '6C.parquet', '6E.parquet', '6J.parquet']

Test file: 6A.parquet
Has usd_risk_volume: True
Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'delivery month', 'open interest', 'daily_prices_change_pts', 'daily_prices_change_percent', 'daily_std_full', 'annualized_std_full', 'ewma32_std', 'rolling_avg_volume_60d', 'rolling_avg_volume_20d', 'annualized_vol', 'usd_risk_volume']


In [19]:
from pathlib import Path
import pandas as pd

# Simulate what the backtest does
def load_instrument_df(filepath, date_col, close_col, vol_col):
    """Mimic the backtest loading function"""
    df = pd.read_parquet(filepath)
    
    print(f"After reading parquet:")
    print(f"  Columns: {df.columns.tolist()}")
    print(f"  Has usd_risk_volume: {'usd_risk_volume' in df.columns}")
    
    # Check if there's any column filtering happening
    df = df[[date_col, close_col, vol_col]]  # This might be the problem!
    
    print(f"After selecting columns [{date_col}, {close_col}, {vol_col}]:")
    print(f"  Columns: {df.columns.tolist()}")
    print(f"  Has usd_risk_volume: {'usd_risk_volume' in df.columns}")
    
    return df

# Test it
test_file = Path("./04a-risk_volume/norgate_continuous/parquet/6A.parquet")
df = load_instrument_df(test_file, "date", "close", "ewma32_std")

After reading parquet:
  Columns: ['date', 'open', 'high', 'low', 'close', 'volume', 'delivery month', 'open interest', 'daily_prices_change_pts', 'daily_prices_change_percent', 'daily_std_full', 'annualized_std_full', 'ewma32_std', 'rolling_avg_volume_60d', 'rolling_avg_volume_20d', 'annualized_vol', 'usd_risk_volume']
  Has usd_risk_volume: True
After selecting columns [date, close, ewma32_std]:
  Columns: ['date', 'close', 'ewma32_std']
  Has usd_risk_volume: False


In [20]:
#!/usr/bin/env python3
import os
from pathlib import Path
import pandas as pd
import numpy as np

"""
===============================================================================
FUTURES BACKTEST ENGINE WITH VOL-TARGET POSITION SIZING (EWMA32)
WITH MICRO/MINI TRADE MAPPING
===============================================================================

Signals/vol/price:
- Always computed from FULL-SIZE "signal symbol" data files.

Sizing + P&L:
- Use MICRO/MINI "trade symbol" multiplier (point_value) when available per mapping.
- Price changes are still taken from the signal symbol (assumes micro tracks underlying).

Position sizing:

    N_t = floor( Equity_t * tau_i / (Multiplier_trade * Price_signal_t * FX * sigma_ann_t) )

Where:
- sigma_ann_t = ewma32_std_daily * sqrt(256)
- tau_i = target_portfolio_vol / K_t (equal risk split across active instruments)
- FX configurable, default 1

P&L:
    pnl_t = sum_i contracts_{i,t-1} * multiplier_trade_i * FX * (close_signal_t - close_signal_{t-1})

Outputs:
- portfolio_equity.(csv|parquet)
- positions.(csv|parquet)              # positions expressed in TRADE contracts (e.g., MES)
- trades.csv                            # contract changes in TRADE contracts
- instrument_map.csv                    # signal_symbol -> trade_symbol -> multiplier
"""

# ============================================================
# Configuration
# ============================================================

# ==================== MICRO/MINI CONTRACT MAPPING ====================
# Maps full-size signal symbol -> micro/mini trade symbol
# Only includes mappings where the micro/mini EXISTS and is commonly traded

MICRO_MINI_MAPPING = {
    # Equity Indices - Micros available in Norgate
    'ES': 'MES',    # E-mini S&P 500 -> Micro E-mini S&P 500
    'NQ': 'MNQ',    # E-mini Nasdaq -> Micro E-mini Nasdaq
    'RTY': 'M2K',   # E-mini Russell 2000 -> Micro E-mini Russell
    'YM': 'MYM',    # E-mini Dow -> Micro E-mini Dow

    # Crypto - Micros available in Norgate
    'BTC': 'MBT',   # Bitcoin -> Micro Bitcoin
    'ETH': 'MET',   # Ether -> Micro Ether
}

# Additional mappings for contracts where micro EXISTS at broker but NOT in Norgate
# These use full-size data for signals but micro specs for sizing/P&L
BROKER_ONLY_MICROS = {
    # Currencies
    '6A': 'M6A',
    '6B': 'M6B',
    '6C': 'MCD',
    '6E': 'M6E',
    '6J': 'MJY',
    '6S': 'MSF',

    # Energy
    'CL': 'MCL',
    'NG': 'MNG',
    'RB': 'MRB',
    'HO': 'MHO',

    # Metals
    'GC': 'MGC',
    'SI': 'SIL',
    'HG': 'MHG',

    # Agriculture
    'ZC': 'MZC',
    'ZW': 'MZW',
    'ZS': 'MZS',
    'ZM': 'MZM',
    'ZL': 'MZL',

    # Volatility
    'VX': 'VXM',
}

# Inverse map: micro/mini trade symbol -> full-size signal symbol
# Used to ensure we do NOT accidentally use micro files (e.g., MES.parquet) as signal sources

MICRO_TO_SIGNAL = {v: k for k, v in MICRO_MINI_MAPPING.items()}

print(f"✓ Micro mappings defined")

CONFIG = {
    # Limit trading universe by TRADE symbol (e.g., {"MES"} to trade only Micro ES)
    #"trade_symbol_allowlist": {"MES"},  # set to None to disable filtering
    "trade_symbol_allowlist": None,  # set to None to disable filtering

    # Backtest range (None means use all available)
    "start_date": "2015-01-01",
    "end_date": None,
 
    # Capital + risk target
    "initial_capital": 400_000.0,
    "target_portfolio_vol": 0.20,     # 20% annualized target (portfolio-level)
    "trading_days_per_year": 256,
    "fx": 1.0,
    
    # NEW: USD Risk Volume Filtering
    "min_usd_risk_volume": 0,  # Minimum USD risk volume to include instrument (0 = all instruments)
                                # Example: 1_000_000 = only instruments with >$1M daily USD risk
                                # Example: 50_000_000 = only instruments with >$50M daily USD risk

    

    # Signal params (trend strategy)
    "fast_sma": 50,
    "slow_sma": 100,

    # Costs (optional)
    "commission_per_contract": 2.50,  # per trade contract changed
    "apply_commissions": True,

    # Data locations
    "contracts_file": Path("./01-futures_universe/futures_contracts_full.parquet"),

    # IMPORTANT:
    # This directory contains the FULL-SIZE signal symbol data with ewma32_std and close
    "signal_vol_prices_dir": Path("./04a-risk_volume/norgate_continuous/parquet"),
    
    # Column names in per-instrument files
    "date_col": "date",
    "close_col": "close",
    "vol_col": "ewma32_std",  # DAILY EWMA stdev of returns (starts after 32 days in your pipeline)

    # Output root
    "output_root": Path("./05-futures_backtest_position_sizing_mapped"),

    # Behavior
    "require_trade_multiplier": True,  # if True, error if mapped trade symbol has no multiplier
    "default_to_signal_multiplier_if_missing": True,  # if trade multiplier missing, use signal multiplier (if allowed)
}

print(f"✓ Micro mappings defined")
print(f"  Norgate micros: {len(MICRO_MINI_MAPPING)}")
print(f"  Broker-only micros: {len(BROKER_ONLY_MICROS)}")

# ============================================================
# Helpers
# ============================================================

def load_contract_multipliers(contracts_path: Path) -> dict:
    """
    Load point_value multipliers from futures_contracts_full.parquet.
    Returns {symbol: point_value}.
    Tries to infer column names robustly.
    """
    if not contracts_path.exists():
        raise FileNotFoundError(f"Contracts file not found: {contracts_path}")

    df = pd.read_parquet(contracts_path)
    df.columns = [str(c).strip().lower() for c in df.columns]

    sym_col_candidates = ["symbol", "ticker", "root", "contract", "instrument"]
    sym_col = next((c for c in sym_col_candidates if c in df.columns), None)
    if sym_col is None:
        raise ValueError(f"Could not find a symbol column in contracts file. Columns: {list(df.columns)}")

    pv_col_candidates = ["point_value", "pointvalue", "multiplier", "contract_multiplier"]
    pv_col = next((c for c in pv_col_candidates if c in df.columns), None)
    if pv_col is None:
        raise ValueError(f"Could not find a point_value column in contracts file. Columns: {list(df.columns)}")

    out = {}
    for _, r in df.iterrows():
        sym = str(r[sym_col]).strip().upper()
        if not sym or sym.lower() == "nan":
            continue
        pv = pd.to_numeric(r[pv_col], errors="coerce")
        if pd.isna(pv) or pv <= 0:
            continue
        out[sym] = float(pv)

    if not out:
        raise RuntimeError("No valid {symbol: point_value} rows found in contracts file.")

    return out


def load_instrument_df(path: Path, date_col: str, close_col: str, vol_col: str) -> pd.DataFrame:
    df = pd.read_parquet(path)
    df.columns = [str(c).strip().lower() for c in df.columns]

    if date_col not in df.columns:
        if isinstance(df.index, pd.DatetimeIndex):
            df = df.reset_index().rename(columns={"index": date_col})
        else:
            raise ValueError(f"Missing '{date_col}' in {path.name}. Columns: {list(df.columns)}")

    df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
    df = df.dropna(subset=[date_col]).sort_values(date_col)

    df[close_col] = pd.to_numeric(df[close_col], errors="coerce")
    df[vol_col] = pd.to_numeric(df[vol_col], errors="coerce")

    return df


def compute_signals(close: pd.Series, fast: int, slow: int) -> pd.Series:
    fast_ma = close.rolling(fast).mean()
    slow_ma = close.rolling(slow).mean()
    sig = pd.Series(0, index=close.index, dtype=int)
    sig[fast_ma > slow_ma] = 1
    sig[fast_ma < slow_ma] = -1
    return sig


def safe_floor(x: float) -> int:
    if not np.isfinite(x) or x <= 0:
        return 0
    return int(np.floor(x))


def map_trade_symbol(signal_symbol: str) -> str:
    """
    Map full-size signal symbol to trade symbol (micro/mini if available).
    Falls back to signal symbol if no mapping exists.
    """
    s = signal_symbol.upper()
    if s in MICRO_MINI_MAPPING:
        return MICRO_MINI_MAPPING[s]
    if s in BROKER_ONLY_MICROS:
        return BROKER_ONLY_MICROS[s]
    return s


# ============================================================
# Backtest Engine
# ============================================================

def run_backtest(cfg: dict) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    date_col = cfg["date_col"]
    close_col = cfg["close_col"]
    vol_col = cfg["vol_col"]

    multipliers = load_contract_multipliers(cfg["contracts_file"])

    signal_dir: Path = cfg["signal_vol_prices_dir"]
    if not signal_dir.exists():
        raise FileNotFoundError(f"Signal vol/price parquet dir not found: {signal_dir}")

    signal_files = sorted(signal_dir.glob("*.parquet"))
    if not signal_files:
        raise RuntimeError(f"No signal parquet files found in: {signal_dir}")

    # Load signals from full-size files; map to trade symbol for multiplier + P&L sizing
    instruments = []
    data_map = {}
    instrument_meta_rows = []

    for p in signal_files:
        signal_sym = p.stem.strip().upper()

        # SAFEGUARD: If this file is a micro/mini symbol (e.g., MES) that maps back to a full-size
        # signal symbol (e.g., ES), skip it as a signal source. We want full-size data for signals.
        if signal_sym in MICRO_TO_SIGNAL:
            # Example: signal_sym == "MES" -> skip, we'll use "ES" file instead
            continue
        trade_sym = map_trade_symbol(signal_sym)

        # OPTION 1: Filter by TRADE symbol allowlist (e.g., {"MES"}).
        # OPTION 1: Filter by TRADE symbol allowlist (e.g., {"MES"}).
        allow = cfg.get("trade_symbol_allowlist")
        if allow is not None and trade_sym not in set(allow):
            continue

        # Need signal data (close, ewma32, usd_risk_volume)
        df = load_instrument_df(p, date_col, close_col, vol_col)
        
        # OPTION 2: Filter by USD risk volume (NEW)
        min_usd_risk = cfg.get("min_usd_risk_volume", 0)
        if min_usd_risk > 0:
            # Check if usd_risk_volume column exists
            if "usd_risk_volume" not in df.columns:
                print(f"  ⚠️  Warning: {signal_sym} missing 'usd_risk_volume' column, skipping")
                print(f"     Make sure you're reading from 04a-risk_volume directory")
                continue
            
            # Get the most recent USD risk value (last non-NaN value)
            usd_risk_series = df["usd_risk_volume"].dropna()
            if len(usd_risk_series) == 0:
                print(f"  ⚠️  {signal_sym}: No valid USD risk data, skipping")
                continue
            
            recent_usd_risk = float(usd_risk_series.iloc[-1])
            
            # Filter out if below minimum
            if recent_usd_risk < min_usd_risk:
                print(f"  🚫 {signal_sym} (trades as {trade_sym}): "
                      f"USD risk ${recent_usd_risk:,.0f} < min ${min_usd_risk:,.0f} - FILTERED OUT")
                continue
            else:
                print(f"  ✅ {signal_sym} (trades as {trade_sym}): "
                      f"USD risk ${recent_usd_risk:,.0f} >= min ${min_usd_risk:,.0f} - INCLUDED")
        
        df = df.set_index(date_col)
        df["signal"] = compute_signals(df[close_col], cfg["fast_sma"], cfg["slow_sma"])

        # Determine multiplier to use (trade symbol preferred)
        trade_mult = multipliers.get(trade_sym)
        signal_mult = multipliers.get(signal_sym)

        chosen_mult = trade_mult

        if chosen_mult is None:
            if cfg.get("default_to_signal_multiplier_if_missing", True) and signal_mult is not None:
                chosen_mult = signal_mult
            elif cfg.get("require_trade_multiplier", True):
                raise RuntimeError(
                    f"Missing multiplier for trade symbol '{trade_sym}' (mapped from '{signal_sym}'). "
                    f"Also signal multiplier exists? {signal_mult is not None}."
                )

        if chosen_mult is None:
            # if both missing, skip
            continue

        data_map[signal_sym] = df
        instruments.append(signal_sym)

        instrument_meta_rows.append({
            "signal_symbol": signal_sym,
            "trade_symbol": trade_sym,
            "multiplier_used": float(chosen_mult),
            "multiplier_source": "trade_symbol" if trade_mult is not None else ("signal_symbol" if signal_mult is not None else "missing"),
        })

    if not instruments:
        raise RuntimeError(
            "No instruments loaded. Check that your signal files exist and multipliers are available."
        )

    instrument_map_df = pd.DataFrame(instrument_meta_rows).sort_values(["signal_symbol", "trade_symbol"])
    print(instrument_map_df)
    # Build common calendar (intersection across signal symbols)
    common_dates = None
    for sym in instruments:
        idx = data_map[sym].index
        common_dates = idx if common_dates is None else common_dates.intersection(idx)
    common_dates = common_dates.sort_values()

    if cfg["start_date"]:
        common_dates = common_dates[common_dates >= pd.Timestamp(cfg["start_date"])]
    if cfg["end_date"]:
        common_dates = common_dates[common_dates <= pd.Timestamp(cfg["end_date"])]

    if len(common_dates) < 3:
        raise RuntimeError("Not enough common dates after filtering to run the backtest.")

    # Matrices
    close = pd.DataFrame({sym: data_map[sym].reindex(common_dates)[close_col] for sym in instruments})
    vol_d = pd.DataFrame({sym: data_map[sym].reindex(common_dates)[vol_col] for sym in instruments})
    sig = pd.DataFrame({sym: data_map[sym].reindex(common_dates)["signal"] for sym in instruments}).astype(int)

    # Annualize daily EWMA vol
    vol_ann = vol_d * np.sqrt(cfg["trading_days_per_year"])

    # Multiplier per signal symbol is actually the "trade multiplier used"
    mult_used = {}
    trade_symbol_used = {}
    for _, r in instrument_map_df.iterrows():
        s = r["signal_symbol"]
        mult_used[s] = float(r["multiplier_used"])
        trade_symbol_used[s] = str(r["trade_symbol"])

    # State
    equity = float(cfg["initial_capital"])
    fx = float(cfg["fx"])
    target_port_vol = float(cfg["target_portfolio_vol"])
    comm = float(cfg["commission_per_contract"])
    apply_comm = bool(cfg["apply_commissions"])

    # Positions held, expressed in TRADE contracts, keyed by signal symbol
    prev_pos = {sym: 0 for sym in instruments}

    # Logs
    portfolio_rows = []
    positions_rows = []
    trades_rows = []

    for t in range(1, len(common_dates)):
        dt_prev = common_dates[t - 1]
        dt = common_dates[t]

        # --- 1) Realize P&L from positions held over (dt_prev -> dt) ---
        pnl = 0.0
        for sym in instruments:
            c0 = close.at[dt_prev, sym]
            c1 = close.at[dt, sym]
            if pd.isna(c0) or pd.isna(c1):
                continue
            mult = mult_used[sym]  # TRADE multiplier
            pnl += prev_pos[sym] * mult * fx * float(c1 - c0)

        equity_before_rebal = equity
        equity += pnl

        # --- 2) Active set K_t for equal risk allocation ---
        active_mask = (
            (sig.loc[dt] != 0)
            & close.loc[dt].notna()
            & vol_ann.loc[dt].notna()
            & (vol_ann.loc[dt] > 0)
        )
        active_syms = active_mask[active_mask].index.tolist()
        K = len(active_syms)

        tau_i = (target_port_vol / K) if K > 0 else 0.0

        # --- 3) Compute target positions for dt -> dt+1 ---
        new_pos = prev_pos.copy()

        for sym in instruments:
            s = int(sig.at[dt, sym])
            c = close.at[dt, sym]
            va = vol_ann.at[dt, sym]
            mult = mult_used[sym]  # TRADE multiplier

            if sym not in active_syms:
                new_pos[sym] = 0
                continue

            denom = mult * float(c) * fx * float(va)
            if denom <= 0 or not np.isfinite(denom):
                new_pos[sym] = 0
                continue

            N_float = equity * tau_i / denom
            N = safe_floor(N_float)
            new_pos[sym] = int(s * N)

        # --- 4) Commissions on changes (in trade contracts) ---
        commissions = 0.0
        if apply_comm:
            for sym in instruments:
                delta = new_pos[sym] - prev_pos[sym]
                if delta != 0:
                    commissions += abs(delta) * comm

        equity -= commissions

        # --- 5) Log trades ---
        for sym in instruments:
            delta = new_pos[sym] - prev_pos[sym]
            if delta != 0:
                trades_rows.append({
                    "date": dt.date().isoformat(),
                    "signal_symbol": sym,
                    "trade_symbol": trade_symbol_used[sym],
                    "prev_contracts": int(prev_pos[sym]),
                    "new_contracts": int(new_pos[sym]),
                    "delta_contracts": int(delta),
                    "commission": float(abs(delta) * comm) if apply_comm else 0.0,
                    "signal": int(sig.at[dt, sym]),
                    "close_signal": float(close.at[dt, sym]) if pd.notna(close.at[dt, sym]) else np.nan,
                    "vol_ann": float(vol_ann.at[dt, sym]) if pd.notna(vol_ann.at[dt, sym]) else np.nan,
                    "multiplier_trade": float(mult_used[sym]),
                    "tau_per_instrument": float(tau_i),
                    "N_float": float(abs(equity * tau_i / (mult_used[sym] * float(close.at[dt, sym]) * fx * float(vol_ann.at[dt, sym]))))
                              if (sym in active_syms and pd.notna(close.at[dt, sym]) and pd.notna(vol_ann.at[dt, sym]) and float(vol_ann.at[dt, sym]) > 0)
                              else np.nan,
                })

        # Positions table: use trade symbols in columns for readability
        pos_row = {"date": dt.date().isoformat()}
        for sym in instruments:
            col = trade_symbol_used[sym]
            pos_row[col] = int(new_pos[sym])
        positions_rows.append(pos_row)

        # Portfolio row
        ret = (equity - equity_before_rebal) / equity_before_rebal if equity_before_rebal != 0 else np.nan
        portfolio_rows.append({
            "date": dt.date().isoformat(),
            "equity": float(equity),
            "pnl": float(pnl),
            "commissions": float(commissions),
            "return": float(ret),
            "K_active": int(K),
            "tau_per_instrument": float(tau_i),
        })

        prev_pos = new_pos

    portfolio_df = pd.DataFrame(portfolio_rows)
    positions_df = pd.DataFrame(positions_rows).fillna(0)

    # Ensure deterministic column order for positions: date then sorted trade symbols
    if not positions_df.empty:
        cols = ["date"] + sorted([c for c in positions_df.columns if c != "date"])
        positions_df = positions_df[cols]

    trades_df = pd.DataFrame(trades_rows)

    return portfolio_df, positions_df, trades_df, instrument_map_df


def write_outputs(cfg: dict, portfolio_df: pd.DataFrame, positions_df: pd.DataFrame,
                  trades_df: pd.DataFrame, instrument_map_df: pd.DataFrame) -> None:
    out_root: Path = cfg["output_root"]
    out_root.mkdir(parents=True, exist_ok=True)

    # portfolio
    portfolio_csv = out_root / "portfolio_equity.csv"
    portfolio_pq  = out_root / "portfolio_equity.parquet"
    portfolio_df.to_csv(portfolio_csv, index=False)
    portfolio_df.to_parquet(portfolio_pq, index=False, compression="snappy")

    # positions (trade contracts)
    positions_csv = out_root / "positions.csv"
    positions_pq  = out_root / "positions.parquet"
    positions_df.to_csv(positions_csv, index=False)
    positions_df.to_parquet(positions_pq, index=False, compression="snappy")

    # trades
    if not trades_df.empty:
        trades_csv = out_root / "trades.csv"
        trades_df.to_csv(trades_csv, index=False)

    # mapping audit
    map_csv = out_root / "instrument_map.csv"
    instrument_map_df.to_csv(map_csv, index=False)

    print(f"Wrote outputs to: {out_root.resolve()}")
    print(f"- {portfolio_csv.name}, {portfolio_pq.name}")
    print(f"- {positions_csv.name}, {positions_pq.name}")
    if not trades_df.empty:
        print(f"- trades.csv")
    print(f"- {map_csv.name}")


# ============================================================
# Main
# ============================================================

def main() -> None:
    portfolio_df, positions_df, trades_df, instrument_map_df = run_backtest(CONFIG)
    write_outputs(CONFIG, portfolio_df, positions_df, trades_df, instrument_map_df)

if __name__ == "__main__":
    main()


✓ Micro mappings defined
✓ Micro mappings defined
  Norgate micros: 6
  Broker-only micros: 19
   signal_symbol trade_symbol  multiplier_used multiplier_source
0             6A          M6A         100000.0     signal_symbol
1             6B          M6B          62500.0     signal_symbol
2             6C          MCD         100000.0     signal_symbol
3             6E          M6E         125000.0     signal_symbol
4             6J          MJY         125000.0     signal_symbol
..           ...          ...              ...               ...
94            ZQ           ZQ           4167.0      trade_symbol
95            ZR           ZR             20.0      trade_symbol
96            ZS          MZS             50.0     signal_symbol
97            ZT           ZT           2000.0      trade_symbol
98            ZW          MZW             50.0     signal_symbol

[99 rows x 4 columns]
Wrote outputs to: C:\TWS API\source\pythonclient\TradingIdeas\Futures\05-futures_backtest_position_siz

In [21]:
from pathlib import Path
import pandas as pd

path = Path(r"C:\TWS API\source\pythonclient\TradingIdeas\Futures\04a-risk_volume\norgate_continuous\parquet\6A.parquet")

df = pd.read_parquet(path)
print("Columns:")
print(df.columns.tolist())
print("\nDtypes:")
print(df.dtypes)

Columns:
['date', 'open', 'high', 'low', 'close', 'volume', 'delivery month', 'open interest', 'daily_prices_change_pts', 'daily_prices_change_percent', 'daily_std_full', 'annualized_std_full', 'ewma32_std', 'rolling_avg_volume_60d', 'rolling_avg_volume_20d', 'annualized_vol', 'usd_risk_volume']

Dtypes:
date                           datetime64[ns]
open                                  float32
high                                  float32
low                                   float32
close                                 float32
volume                                float32
delivery month                        float32
open interest                         float32
daily_prices_change_pts               float32
daily_prices_change_percent           float32
daily_std_full                        float64
annualized_std_full                   float64
ewma32_std                            float64
rolling_avg_volume_60d                float64
rolling_avg_volume_20d                float64
annu